<a href="https://colab.research.google.com/github/anaswaraku/Kidney-disease-detection/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet import ResNet50
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define ResNet model
def build_resnet(input_shape, num_classes):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
# Define input shape and number of classes
input_shape = (224, 224, 3)  # Adjust according to your image size
num_classes = 4  # Number of disease classes

# Build the model
model = build_resnet(input_shape, num_classes)

In [25]:

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Define paths to training and validation data
train_data_dir = '/content/drive/MyDrive/Preprocessed/Preprocessed/train'
validation_data_dir = '/content/drive/MyDrive/Preprocessed/Preprocessed/validation'

In [26]:
# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
test_data_dir = '/content/drive/MyDrive/Preprocessed/Preprocessed/test'
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'

)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 346 images belonging to 4 classes.
Found 480 images belonging to 4 classes.
Found 285 images belonging to 4 classes.


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    epochs=10,  # Adjust as needed
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // val_generator.batch_size)

# Save the model
model.save('resnet_model.h5')

Epoch 1/10
 3/15 [=====>........................] - ETA: 5:50 - loss: 9.6434 - accuracy: 0.2604